In [620]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time


In [621]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Executa sem abrir o navegador
chrome_options.add_argument("--disable-gpu")  # Evita alguns bugs gráficos no headless
chrome_options.add_argument("--window-size=1920x1080")  # Garante renderização correta

# Inicializa o WebDriver do Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)


In [622]:
# trocar o link para trocar cidades, funiona para qualquer cidade
# ideia: criar uma lista com links de diversas cidades e rodar o programa todo
url = "https://www.quintoandar.com.br/comprar/imovel/rio-de-janeiro-rj-brasil?referrer=home&profiling=true"
driver.get(url)
main_window = driver.current_window_handle

# Aguarda alguns segundos para carregar o JavaScript (ajuste conforme necessário)
time.sleep(2)

In [623]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Aumentar o range para aumentar o numero de imoveis
for i in range(1):
    see_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "see-more"))
    )

    # Clica no botão
    see_more_button.click()




In [624]:
# recupera os cards dos imoveis
cards = driver.find_elements(By.CLASS_NAME, "StyledLink_styledLink__P_6FN")

print(f"Total de imóveis encontrados: {len(cards)}")

Total de imóveis encontrados: 12


In [625]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

#### Extração dos dados

In [626]:
precos = []
num_quartos = []
num_banheiros = []
num_vagas = []
m_quadrados = []
tempos_publi = []
itens_disponiveis = []
itens_indisponiveis = []
tipos_imoveis = []
cidades = []
regioes = []


In [627]:
for card in cards:
    href = card.get_attribute("href")
    
    html = requests.get(href).content
    soup = BeautifulSoup(html, 'html.parser')
    
    preco = soup.find('div', {'class': 'PriceTableSale_price__dexUu'})
    infos = soup.find_all('div', {'class': 'MuiBox-root mui-15au7ed'})
    spans = soup.find_all('span')
    tipo_imovel = soup.find_all('h1')

    
    # a lista de disponiveis fica no indice 0, e de indisponiveis no indice 1
    itens = soup.find_all('div', {'class': 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-6 mui-1s50f5r'})
    
    # tratamento, pois nem todos imoveis possuem itens disponiveis
    try:
        itens_disponivels_pagina = itens[0].find_all('div', {'class': 'AmenitiesList_itemsWrapper__PLY3c'})
        itens_indisponivels_pagina = itens[1].find_all('div', {'class': 'AmenitiesList_itemsWrapper__PLY3c'})
    except IndexError:
        itens_disponiveis.append(np.nan)
        itens_indisponiveis.append(np.nan)
        
    
    # cria vetores aux para itens disponiveis e indisponiveis de cada umovel
    disponivel_aux = []
    indisponivel_aux = []

    for item in itens_disponivels_pagina:
        try:
            disponivel_aux.append(item.text)
        except IndexError:
                disponivel_aux.append(np.nan)
                
    for item in itens_indisponivels_pagina:
        try:
            indisponivel_aux.append(item.text)
        except IndexError:
            indisponivel_aux.append(np.nan)
            
    # insere o vetor de itens relacionados ao imovel
    itens_disponiveis.append(disponivel_aux)
    itens_indisponiveis.append(indisponivel_aux)
    
    # o tipo do imovelsempre éo primeiro h1, então é valido fazer
    tipos_imoveis.append(tipo_imovel[0].text)
    
    # o período de publicação é sempre um span, e sempre esta no índice 106, então é possível fazer a seguinte operação
    tempos_publi.append(spans[106].text)
    
    # cidade e regiao sao sempre spans, na posição 100 e 101, entao é valido
    cidades.append(spans[100].text)
    regioes.append(spans[101].text)
    
    precos.append(preco.text)
    
    try:
        num_quartos.append(infos[0].text)
        m_quadrados.append(infos[1].text)
        num_vagas.append(infos[3].text)
        num_banheiros.append(infos[4].text)
    except IndexError:
        num_quartos.append(np.nan)
        m_quadrados.append(np.nan)
        num_vagas.append(np.nan)
        num_banheiros.append(np.nan)
    
    

#### Processamento dos dados

In [628]:
for i, tipo_imovel in enumerate(tipos_imoveis):
    tipos_imoveis[i] = tipo_imovel.split()[0] 

In [629]:
for i, quarto in enumerate(num_quartos):
    num_quartos[i] = quarto.split()[0] 

In [630]:
for i, tamanho in enumerate(m_quadrados):
    if isinstance(tamanho, str) and tamanho.strip():  
        m_quadrados[i] = tamanho.split()[0]
    else:
        m_quadrados[i] = str(tamanho).split()[0] 

In [631]:
for i, vaga in enumerate(num_vagas):
    num_vagas[i] = vaga.split()[0]

In [632]:
for i, banheiro in enumerate(num_banheiros):
    num_banheiros[i] = banheiro.split()[0]

In [633]:
for i, preco in enumerate(precos):
    partes = preco.split('\xa0', 1)
    precos[i] = partes[1]

In [634]:
# função que converte os  tempos de publicação em datas
import dateparser
from datetime import datetime

def converter_para_data(data_string):
    # Remover o prefixo 'Publicado há' para ficar com apenas a parte de tempo relativo
    data_string = data_string.replace('Publicado há', '').strip()

    # Usando o dateparser para analisar a string com a data relativa
    data_convertida = dateparser.parse(data_string, settings={'PREFER_DATES_FROM': 'past'})

    # Verificando se a conversão foi bem-sucedida
    if data_convertida:
        # Retorna a data no formato "dia/mês/ano"
        return data_convertida.strftime('%d/%m/%Y')
    else:
        # Caso não consiga converter, retorna uma mensagem de erro
        return "Data inválida"

In [635]:
datas_completas = [converter_para_data(data) for data in tempos_publi]

In [636]:
import datetime
import calendar

def extrair_informacoes(data_string):
    # Converte a string para o formato datetime
    data_convertida = datetime.datetime.strptime(data_string, '%d/%m/%Y')
    
    # Extraindo as informações solicitadas
    dia = data_convertida.day
    mes = data_convertida.month
    ano = data_convertida.year
    dia_da_semana = data_convertida.weekday()  # Retorna 0 para segunda-feira até 6 para domingo
    dia_da_semana_extenso = [
        'Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 
        'Sexta-feira', 'Sábado', 'Domingo'
    ][dia_da_semana]  # Nome do dia da semana em português
    semana_ano = data_convertida.isocalendar()[1]  # Número da semana do ano
    quarter = (mes - 1) // 3 + 1  # Determina o trimestre (Q1, Q2, Q3 ou Q4)

    # Retorna as informações como um dicionário
    return {
        'Dia': dia,
        'Mes': mes,
        'Ano': ano,
        'Dia da semana (número)': dia_da_semana,
        'Dia da semana (extenso)': dia_da_semana_extenso,
        'Semana do ano': semana_ano,
        'Quarter': f'Q{quarter}'
    }

In [637]:
print(extrair_informacoes(datas_completas[0]))

{'Dia': 9, 'Mes': 2, 'Ano': 2025, 'Dia da semana (número)': 6, 'Dia da semana (extenso)': 'Domingo', 'Semana do ano': 6, 'Quarter': 'Q1'}


In [638]:
dia = []
mes = []
ano = []
dia_da_semana = []
dia_da_semana_extenso = []
semana_do_ano = []
quarter = []

In [639]:
for data in datas_completas:
    informacoes = extrair_informacoes(data)
    
    # Acessando as chaves do dicionário para preencher as listas
    dia.append(informacoes['Dia'])
    mes.append(informacoes['Mes'])
    ano.append(informacoes['Ano'])
    dia_da_semana.append(informacoes['Dia da semana (número)'])
    dia_da_semana_extenso.append(informacoes['Dia da semana (extenso)'])
    semana_do_ano.append(informacoes['Semana do ano'])
    quarter.append(f'{informacoes["Quarter"]}')

* Estado terá que ser inserido manualmente para cada cidade

In [640]:
dados = pd.DataFrame({'Preco':precos})
dados['Num Quartos'] = num_quartos
dados['Num Banheiros'] =num_banheiros
dados['Num Vagas'] = num_vagas
dados['Tamanho'] = m_quadrados
dados['Tipo'] = tipos_imoveis
dados['Cidade'] = cidades
dados['Regiao'] = regioes
dados['Itens Disponivels'] = itens_disponiveis
dados['Itens Indisponíveis'] = itens_indisponiveis
dados['Data Completa'] = datas_completas
dados['Dia'] = dia
dados['Mes'] = mes
dados['Ano'] = ano
dados['Dia da Semana'] = dia_da_semana
dados['Dia da Semana Extenso'] = dia_da_semana_extenso
dados['Semana do Ano'] = semana_do_ano
dados['Quarter'] = quarter

In [641]:
dados

,Preco,Num Quartos,Num Banheiros,Num Vagas,Tamanho,Tipo,Cidade,Regiao,Itens Disponivels,Itens Indisponíveis,Data Completa,Dia,Mes,Ano,Dia da Semana,Dia da Semana Extenso,Semana do Ano,Quarter
0,250.000,2,3,1,145,Casa,Rio de Janeiro,Tanque,"[Box, Varanda, Armários no quarto, Armários no...","[Banheira de hidromassagem, Piscina privativa,...",09/02/2025,9,2,2025,6,Domingo,6,Q1
1,250.000,4,3,3,150,Casa,Rio de Janeiro,Piedade,"[Box, Varanda, Piscina privativa, Armários no ...","[Banheira de hidromassagem, Armários na cozinh...",16/02/2025,16,2,2025,6,Domingo,7,Q1
2,300.000,3,3,1,180,Casa,Rio de Janeiro,Tanque,"[Box, Varanda, Armários no quarto, Armários no...","[Banheira de hidromassagem, Piscina privativa,...",08/02/2025,8,2,2025,5,Sábado,6,Q1
3,175.000,2,1,1,50,Apartamento,Rio de Janeiro,Taquara,"[Box, Varanda, Armários no quarto, Armários no...","[Banheira de hidromassagem, Piscina privativa,...",10/02/2025,10,2,2025,0,Segunda-feira,7,Q1
4,205.000,2,1,1,52,Apartamento,Rio de Janeiro,Taquara,"[Box, Varanda, Armários no quarto, Armários no...","[Banheira de hidromassagem, Piscina privativa,...",15/02/2025,15,2,2025,5,Sábado,7,Q1
5,250.000,3,2,-,120,Casa,Rio de Janeiro,Taquara,[Varanda],"[Banheira de hidromassagem, Box, Piscina priva...",24/01/2025,24,1,2025,4,Sexta-feira,4,Q1
6,170.000,2,1,1,48,Apartamento,Rio de Janeiro,Taquara,"[Banheira de hidromassagem, Varanda, Armários ...","[Box, Piscina privativa, Apartamento cobertura...",16/01/2025,16,1,2025,3,Quinta-feira,3,Q1
7,165.000,2,1,1,57,Apartamento,Rio de Janeiro,Jacarepaguá,"[Box, Armários nos banheiros, Armários na cozi...","[Banheira de hidromassagem, Varanda, Piscina p...",16/01/2025,16,1,2025,3,Quinta-feira,3,Q1
8,212.000,2,2,1,50,Apartamento,Rio de Janeiro,Taquara,"[Cozinha americana, Área de serviço]","[Banheira de hidromassagem, Box, Varanda, Pisc...",15/02/2025,15,2,2025,5,Sábado,7,Q1
9,250.000,3,3,-,120,Casa,Rio de Janeiro,Taquara,"[Box, Varanda, Armários no quarto, Armários no...","[Banheira de hidromassagem, Piscina privativa,...",05/02/2025,5,2,2025,2,Quarta-feira,6,Q1


In [642]:
driver.quit()